In [12]:
import pandas
import scipy.stats as stats
import networkx
import seaborn
import pyreadstat

In [2]:
df_original, meta_original = pyreadstat.read_sav("data-covid-source.sav")
print(f"There're {len(df_original.columns)} variables in the data:")
pandas.set_option('display.max_columns', None)
# pandas.set_option("display.max_rows", None)
pandas.set_option("display.precision", 1)
display(df_original.describe(percentiles=[]).transpose())
pandas.reset_option("max_columns")
# pandas.reset_option("max_rows")
pandas.reset_option("precision")

There're 510 variables in the data:


,count,mean,std,min,50%,max
Id,547.0,400.4,234.9,1.0,389.0,832.0
ATT_1_1,547.0,3.0,0.0,3.0,3.0,3.0
ATT_1_2,547.0,6.0,0.0,6.0,6.0,6.0
TIME_1,547.0,2026.2,3907.8,221.0,1274.0,55552.0
TIME_2,547.0,2363.2,5401.8,264.0,1408.0,88447.0
...,...,...,...,...,...,...
Wiedza_koronavir_t5,547.0,-0.3,0.2,-0.6,-0.3,0.2
WHO_FIZ_t4,547.0,5.9,1.7,1.0,6.0,9.0
WHO_SPOL_t4,547.0,6.2,1.7,1.0,6.5,9.0
BOGARDUS_uchodz_t4,547.0,2.7,1.1,1.0,2.6,5.0


In the above table there's some basic information on 510 variables measuring various psychological metrics from a longitude study of 547 people.<br>
There were 5 measurements, each one week apart from the others.<br>
The variables encode the number of measurement in their name by appending "\_t2", \_t3", ... to their name. For the first measurement "\_t1" is sometimes appended, sometimes not, depending on the variable.<br>
For some variables, not all 5 measurements are present - for example for communal narcissm ("cni" in the source data) the measurements are from week 1 and week 5 only. In the "cni" example, there are 16 variables from "cni1" to "cni16" describing various 16 metrics of communal narcissm, and are followed up with "cni1_t5" to "cni16_t5" (where "\_t5" indicates the week no. 5 of measurement taken).

The goal of this excercise is to:<br>
1. perform paired sample t-test for following measurements;
1. compute [Cohen's d - effect size value](https://en.wikipedia.org/wiki/Effect_size#Cohen's_d);
1. plot Cohen's d change in time (with mean trajectories);
1. compute partial correlation networks and basic networks' metrics;
1. plot the networks (with only signifcant changes - uncorrected p>0.05);
1. plot correlogram for average absolute edge values between and within questionnaires;
1. plot basic graphs' metrics for particular measurements at different times to easily observe change (or lack of thereof);
1. compute and show in correlation matrix the Pearson's correlation of considered variables

Only selected variables are to be considered, and those are:
1. Knowledge ("wiedza");
1. Communal Narcissim ("cni");
1. Dissinfection ("odkazanie");
1. Stay-at-home ("siedz_w_domu");
1. Cleaning-hands ("mycie_rak");
1. Masks ("maseczki");
1. Threatened ("zagr_ja");
1. Dark Triad ("DT")
1. Series of WHO health metrics ("WHO_zdr", "WHO_sen", "WHO_samooc_ind", "WHO_zwiazki", "WHO_wsparcie");


In [10]:
desired_variable_names = ['cni', 'DT', 'wiedza', 'odkazanie', 'siedz_w_domu', 'mycie_rak', 'maseczki', 'zagr_ja', 'WHO']
variable_list = []

for column in df_original.columns:
    if any(d in column for d in desired_variable_names):
        variable_list.append(column)
variable_list.sort(reverse=True)

pandas.set_option('display.max_columns', None)
pandas.set_option("display.max_rows", None)
pandas.set_option("display.precision", 1)
display(df_original[variable_list].describe(percentiles=[]).transpose())
pandas.reset_option("max_columns")
pandas.reset_option("max_rows")
pandas.reset_option("precision")

,count,mean,std,min,50%,max
zagr_ja_t4,547.0,3.1,1.0,1.0,3.0,5.0
zagr_ja,547.0,3.4,1.0,1.0,3.0,5.0
wiedza9_t5,547.0,1.8,0.4,1.0,2.0,2.0
wiedza9,547.0,0.2,0.4,0.0,0.0,1.0
wiedza8_t5,547.0,1.9,0.3,1.0,2.0,2.0
wiedza8,547.0,0.1,0.4,0.0,0.0,1.0
wiedza7_t5,547.0,1.4,0.5,1.0,1.0,2.0
wiedza7,547.0,0.2,0.4,0.0,0.0,1.0
wiedza6_t5,547.0,1.6,0.5,1.0,2.0,2.0
wiedza6,547.0,0.4,0.5,0.0,0.0,1.0


Let's match variable names from first study with second (week 4 or week 5) study:

In [15]:
varaibles_time_pairs_dict = dict()
for variable_candidate in variable_list:
    for match_candidate in variable_list:
        if variable_candidate is not match_candidate:
            if variable_candidate in match_candidate:
                varaibles_time_pairs_dict[variable_candidate] = match_candidate

# Let me correct a few wrong matches manually:
varaibles_time_pairs_dict['siedz_w_domu'] = 'siedz_w_domu_t4'
varaibles_time_pairs_dict['maseczki'] = 'maseczki_t4'
varaibles_time_pairs_dict['cni1'] = 'cni1_t5' 
varaibles_time_pairs_dict['DT1'] = 'DT1_t5'

For each of paired weeks of measurement for given variable, let's calculate the paired sample t-test:

In [26]:
paired_tests_dict = dict()
for var_week_1, var_week_2 in varaibles_time_pairs_dict.items():
    pre = df_original[var_week_1]
    post = df_original[var_week_2]
    paired_tests_dict[f"{var_week_1}-{var_week_2}"] = stats.ttest_rel(pre, post)

print("Variable pairs for which a test for the null hypothesis that two \n(related as mentioned above) samples have identical average values holds \n(null hypothesis is NOT rejected):\n")
for var_pair, result_obj in paired_tests_dict.items():
    if result_obj.pvalue >= 0.05:
        print(f"{var_pair:33} - p-value: {result_obj.pvalue:.3f}")

print("\nVariable pairs for which a test for the null hypothesis that two \nsamples have identical average values fails \n(null hypothesis IS rejected):\n")
for var_pair, result_obj in paired_tests_dict.items():
    if result_obj.pvalue < 0.05:
        print(f"{var_pair:33} - p-value: {result_obj.pvalue:.3f}, statistic: {result_obj.statistic:.3f}")

Variable pairs for which a test for the null hypothesis that two 
(related as mentioned above) samples have identical average values holds 
(null hypothesis is NOT rejected):

cni9-cni9_t5                      - p-value: 0.721
cni8-cni8_t5                      - p-value: 0.118
cni7-cni7_t5                      - p-value: 0.422
cni5-cni5_t5                      - p-value: 1.000
cni4-cni4_t5                      - p-value: 0.162
cni3-cni3_t5                      - p-value: 0.075
cni16-cni16_t5                    - p-value: 0.485
cni14-cni14_t5                    - p-value: 1.000
cni11-cni11_t5                    - p-value: 0.754
WHO_zwiazki-WHO_zwiazki_t4        - p-value: 0.583
WHO_wsparcie-WHO_wsparcie_t4      - p-value: 0.734
WHO_sen-WHO_sen_t4                - p-value: 0.223
WHO_samooc_kol-WHO_samooc_kol_t4  - p-value: 0.071
WHO_samooc_ind-WHO_samooc_ind_t4  - p-value: 0.247
WHO_SPOL-WHO_SPOL_t4              - p-value: 0.566
NAR_DT-NAR_DT_t5                  - p-value: 0.833
DT8-DT8_

Let's calculate the Cohen's d value - the difference between two means divided by a standard deviation for the data.<br>
$d = \frac{\bar{x}_1 - \bar{x}_2}{s}$
, where s is [pooled variance](https://en.wikipedia.org/wiki/Pooled_variance):<br>
$s = \sqrt{\frac{(n_1 - 1)s^2_1 + (n_2 - 1)s^2_2}{n_1 + n_2 - 2}}$
<br>
and where $s^2 = \frac{1}{n  - 1}\sum_{i=1}^{n}(x_i - \bar{x})^2$

In [40]:
from functools import reduce
import statistics

d_value_per_pair_dict = dict()
s_sqr = lambda l, a: (1/(len(l) - 1)) * ((sum(map(lambda x: x - a, l))) ** 2)

for var_week_1, var_week_2 in varaibles_time_pairs_dict.items():
    s_sqr_week_1 = s_sqr(df_original[var_week_1], statistics.mean(df_original[var_week_1]))
    s_sqr_week_2 = s_sqr(df_original[var_week_2], statistics.mean(df_original[var_week_2]))
